In [1]:
import pandas as pd 
import sqlite3
import os 

In [2]:
print(os.listdir())

['.ipynb_checkpoints', 'output', 'processed', 'raw', 'Untitled.ipynb', 'Untitled1.ipynb']


In [3]:
df = pd.read_csv("raw/retail_sales_dataset.csv")
print("Original Shape:", df.shape)
df.head()


Original Shape: (1000, 9)


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [5]:
before_rows = df.shape[0]
df = df.drop_duplicates()
after_rows = df.shape[0]
print("Rows before removing duplicates:", before_rows)
print("Rows after removing duplicates:", after_rows)
print("Duplicates removed:", before_rows - after_rows)

Rows before removing duplicates: 1000
Rows after removing duplicates: 1000
Duplicates removed: 0


In [6]:
df.isnull().sum()

Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

In [8]:
df.columns = df.columns.str.lower().str.replace(" ", "_")
df.columns

Index(['_t_r_a_n_s_a_c_t_i_o_n___i_d_', '_d_a_t_e_', '_c_u_s_t_o_m_e_r___i_d_',
       '_g_e_n_d_e_r_', '_a_g_e_', '_p_r_o_d_u_c_t___c_a_t_e_g_o_r_y_',
       '_q_u_a_n_t_i_t_y_', '_p_r_i_c_e___p_e_r___u_n_i_t_',
       '_t_o_t_a_l___a_m_o_u_n_t_'],
      dtype='object')

In [9]:
df.columns = [
    "transaction_id",
    "date",
    "customer_id",
    "gender",
    "age",
    "product_category",
    "quantity",
    "price_per_unit",
    "total_amount"
]

df.columns


Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')

In [10]:
df.dtypes

transaction_id       int64
date                object
customer_id         object
gender              object
age                  int64
product_category    object
quantity             int64
price_per_unit       int64
total_amount         int64
dtype: object

In [11]:
df['date'] = pd.to_datetime(df['date'])
df['price_per_unit'] = df['price_per_unit'].astype(float)
df['total_amount'] = df['total_amount'].astype(float)
df.dtypes


transaction_id               int64
date                datetime64[ns]
customer_id                 object
gender                      object
age                          int64
product_category            object
quantity                     int64
price_per_unit             float64
total_amount               float64
dtype: object

In [12]:
df['calculated_total'] = df['quantity'] * df['price_per_unit']
df['high_value_flag'] = df['total_amount'].apply(lambda x: 1 if x > 500 else 0)
df.head()

,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,calculated_total,high_value_flag
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50.0,150.0,150.0,0
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500.0,1000.0,1000.0,1
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30.0,30.0,30.0,0
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500.0,500.0,500.0,0
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50.0,100.0,100.0,0


In [13]:
df.to_csv("processed/processed_data.csv", index=False)
print("Processed data saved successfully")

Processed data saved successfully


In [14]:
customers = df[['customer_id', 'gender', 'age']].drop_duplicates()
customers.to_csv("output/cusomers.csv", index=False)
print("Customers table created:", customers.shape)

Customers table created: (1000, 3)


In [15]:
transactions = df[['transaction_id', 'date', 'customer_id',
                   'product_category', 'quantity',
                   'price_per_unit', 'total_amount',
                   'high_value_flag']]
transactions.to_csv("output/transactions.csv", index=False)
print("Transactions table created:", transactions.shape)

Transactions table created: (1000, 8)


In [16]:
conn = sqlite3.connect("database.sqlite")
customers.to_sql("customers", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)
conn.close()
print("Data loaded into SQLite successfully")

Data loaded into SQLite successfully


In [17]:
print("Original Rows:", len(df))
print("Customers Rows:", len(customers))
print("Transactions Rows:", len(transactions))

Original Rows: 1000
Customers Rows: 1000
Transactions Rows: 1000
